<a href="https://colab.research.google.com/github/sridhartroy/AIML/blob/main/BigData_Assignment_1B_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.5.6 spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.6-py2.py3-none-any.whl size=317895798 sha256=02e1d48aee860f60e27a343e5ac4a1a20f95831158afa275ec79a04fc613711f
  Stored in directory: /root/.cache/pip/wheels/64/62/f3/ec15656ea4ada0523cae62a1827fe7beb55d3c8c87174aad4a
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 124.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [1]:
import spacy
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col
import re, math, argparse
from pyspark.ml.feature import StopWordsRemover
from operator import add
import math
from collections import Counter

# Initialize Spark session
spark = SparkSession.builder \
    .appName("NER-WordCount") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
!wget -O MovieSummaries.tar.gz http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz

--2025-09-28 05:20:26--  http://www.cs.cmu.edu/~ark/personas/data/MovieSummaries.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48002242 (46M) [application/x-gzip]
Saving to: ‘MovieSummaries.tar.gz’

MovieSummaries.tar. 100%[===================>]  45.78M  4.87MB/s    in 9.7s    

2025-09-28 05:20:36 (4.74 MB/s) - ‘MovieSummaries.tar.gz’ saved [48002242/48002242]



In [10]:
!tar -xzf MovieSummaries.tar.gz
!ls -lh
!ls -lh MovieSummaries


total 46M
drwxrwxr-x 2 14584 14584 4.0K May  6  2013 MovieSummaries
-rw-r--r-- 1 root  root   46M Jun 11  2015 MovieSummaries.tar.gz
drwxr-xr-x 1 root  root  4.0K Sep 25 13:37 sample_data
total 128M
-rw-r--r-- 1 14584 14584  40M May  6  2013 character.metadata.tsv
-rw-r--r-- 1 14584 14584  16M May  6  2013 movie.metadata.tsv
-rw-rw-r-- 1 14584 14584  64K May  6  2013 name.clusters.txt
-rw-rw-r-- 1 14584 14584  73M May  6  2013 plot_summaries.txt
-rw-r--r-- 1 14584 14584 2.6K May  6  2013 README.txt
-rw-rw-r-- 1 14584 14584  57K May  6  2013 tvtropes.clusters.txt


In [17]:
raw = sc.textFile("MovieSummaries/plot_summaries.txt")

movie_names_df0 = spark.read.csv("MovieSummaries/movie.metadata.tsv", sep="\t", header=False)
movie_names_df0.show(5)

movie_names_df = movie_names_df0.selectExpr("_c0 as movie_id", "_c2 as movie_name")
movie_names_df.show(5)


# Ensure doc_id is string to match TF-IDF DF
movie_names_df = movie_names_df.withColumn("movie_id", col("movie_id").cast("string"))
movie_names_df.show(5)

+--------+----------+--------------------+----------+--------+-----+--------------------+--------------------+--------------------+
|     _c0|       _c1|                 _c2|       _c3|     _c4|  _c5|                 _c6|                 _c7|                 _c8|
+--------+----------+--------------------+----------+--------+-----+--------------------+--------------------+--------------------+
|  975900| /m/03vyhn|      Ghosts of Mars|2001-08-24|14010832| 98.0|{"/m/02h40lc": "E...|{"/m/09c7w0": "Un...|{"/m/01jfsb": "Th...|
| 3196793| /m/08yl5d|Getting Away with...|2000-02-16|    NULL| 95.0|{"/m/02h40lc": "E...|{"/m/09c7w0": "Un...|{"/m/02n4kr": "My...|
|28463795|/m/0crgdbh|         Brun bitter|      1988|    NULL| 83.0|{"/m/05f_3": "Nor...|{"/m/05b4w": "Nor...|{"/m/0lsxr": "Cri...|
| 9363483|/m/0285_cd|    White Of The Eye|      1987|    NULL|110.0|{"/m/02h40lc": "E...|{"/m/07ssc": "Uni...|{"/m/01jfsb": "Th...|
|  261236| /m/01mrr1|   A Woman in Flames|      1983|    NULL|106.0|{"/m/043

In [18]:
raw.take(2)

["23890098\tShlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",
 '31186339\tThe nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at special academie

In [9]:
movie_rdd = raw.map(lambda line: line.split("\t", 1)).filter(lambda x: len(x) == 2)
movie_rdd.take(2)

<class 'pyspark.rdd.RDD'>


[['23890098',
  "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all."],
 ['31186339',
  'The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at speci

In [19]:
stop_set = set(StopWordsRemover().getStopWords())  # use Spark's English list
TOKEN_RE = re.compile(r"[A-Za-z][A-Za-z']{1,}")

def tokenize(text):
    return [t.lower() for t in TOKEN_RE.findall(text.lower()) if t.lower() not in stop_set]

In [20]:
tokenize("hard-working")

['hard', 'working']

In [21]:
movie_rdd = movie_rdd.map(lambda x: (x[0], tokenize(x[1]))).filter(lambda x: len(x[1]) > 0)
movie_rdd.take(1)

[('23890098',
  ['shlykov',
   'hard',
   'working',
   'taxi',
   'driver',
   'lyosha',
   'saxophonist',
   'develop',
   'bizarre',
   'love',
   'hate',
   'relationship',
   'despite',
   'prejudices',
   'realize',
   'different'])]

In [22]:
# doc_rdd : (docId, [tokens...])  # tokens AFTER stopword removal/tokenize
# 1) document lengths (|d|) = number of tokens per doc
movie_len = (movie_rdd.map(lambda d: (d[0], len(d[1]))).cache())
print(movie_len.take(2))

# 2) raw TF: ((term, doc), count)
tf_raw = (movie_rdd.flatMap(lambda d: [((term, d[0]), 1) for term in d[1]]).reduceByKey(add))
print(tf_raw.take(2))

#tf_raw1 = tf_raw.map(lambda x: (x[0][1], (x[0][0], x[1])))
#print(tf_raw1.take(2))

#tf_raw2 = tf_raw1.join(movie_len)
#print(tf_raw2.take(2))
# 3) normalize TF by document length: tf_norm = tf / |d|
#    -> ((term, doc), tf_norm)
tf = (tf_raw
      .map(lambda x: (x[0][1], (x[0][0], x[1])))        # key by doc: (doc, (term, tf))
      .join(movie_len)                                     # (doc, ((term, tf), |d|))
      .map(lambda x: ((x[1][0][0], x[0]), x[1][0][1] / float(x[1][1]))))

print(tf.take(2))

[(('shlykov', '23890098'), 1), (('saxophonist', '23890098'), 1)]
[(('six', '20663735'), 0.006993006993006993), (('classmate', '20663735'), 0.006993006993006993)]


In [23]:
df = (tf.map(lambda x: (x[0][0], x[0][1]))  # (term, doc)
        .map(lambda x: (x[0], 1)) # ('six', 1) ('six', 1) ('saxophonist', 1) ('saxophonist', 1)
        .reduceByKey(add))
print(df.take(2))

[('six', 1349), ('madhavan', 50)]


In [24]:
N = movie_rdd.count()
# idf(term) = log((N+1)/(df+1)) + 1  (smoothed)
idf = df.mapValues(lambda dfv: math.log((N) / (dfv)))
idf_map = dict(idf.collect())
idf_bc = sc.broadcast(idf_map)
list(idf_map.items())[:2]

[('six', 3.44556534273934), ('madhavan', 6.7406611935189495)]

In [25]:
tfidf = tf.map(lambda x: (x[0], x[1] * idf_bc.value.get(x[0][0], 0.0)))
tfidf.take(2)
flat_rdd = tfidf.map(lambda x: (x[0][0], x[0][1], float(x[1])))
df = flat_rdd.toDF(["word", "movie_id", "tfidf"])
df.show(5)

+----------+--------+--------------------+
|      word|movie_id|               tfidf|
+----------+--------+--------------------+
|       six|20663735| 0.02409486253663874|
| classmate|20663735| 0.03431105600284575|
|       son|20663735|0.019438880743827568|
|karunakara|20663735| 0.03724714754876607|
|    framed|20663735|0.017144374870765183|
+----------+--------+--------------------+
only showing top 5 rows



In [26]:
def top_docs_for_word_with_name( query_word, top_k=10):
    #print(df.show())
    # Filter for the word
    filtered_df = df.filter(F.col("word") == query_word)
    #print(filtered_df.show())
    # Join with doc_names_df to get names
    joined_df = filtered_df.join(movie_names_df, on="movie_id", how="left")
    #print(joined_df.orderBy(F.col("tfidf").desc().show()))
    # Order by tfidf descending
    top_docs = (joined_df
                .orderBy(F.col("tfidf").desc())
                .select("movie_name")
                .limit(top_k))
    return top_docs

In [27]:
# You already have: tfidf  # RDD[((term, doc_id), tfidf_weight)]
tfidf_df = (tfidf
            .map(lambda x: (x[0][1], x[0][0], float(x[1])))
            .toDF(["movie_id", "word", "tfidf"])
            .cache())
print(tfidf_df.show())

doc_norm_df = (tfidf_df
    .groupBy("movie_id")
    .agg(F.sqrt(F.sum(F.col("tfidf") * F.col("tfidf"))).alias("d_norm"))
    .cache())
print(doc_norm_df.show())

+--------+----------+--------------------+
|movie_id|      word|               tfidf|
+--------+----------+--------------------+
|20663735|       six| 0.02409486253663874|
|20663735| classmate| 0.03431105600284575|
|20663735|       son|0.019438880743827568|
|20663735|karunakara| 0.03724714754876607|
|20663735|    framed|0.017144374870765183|
|20663735| manapally|  0.2607300328413625|
|20663735|  madhavan| 0.04713749086376888|
|20663735|      dysp|0.030443265908712526|
|20663735|      take|0.005376762278935563|
|20663735|   revenge|0.010018539064553827|
|20663735|  idealist|0.024917215644513758|
|20663735|     judge|0.014660069036956505|
|20663735|corruption|0.017303852182835297|
|20663735|  achieved| 0.02065647577127219|
|20663735|      rank|0.020174413553708827|
|20663735|   service|0.025315104460972654|
|20663735|     loses|0.010928070907598992|
|20663735|     learn|0.010318293133933112|
|20663735|  actually|0.009364165576282753|
|20663735|    passes| 0.02423095828553944|
+--------+-

In [28]:
def run_multi_df(query, top_k=10):
    #print(idf_map)
    # keep only words that exist in your vocabulary
    q_tokens = [w for w in tokenize(query) if w in idf_map]
    if not q_tokens:
        return spark.createDataFrame([], "movie_name string, score double")
    #print(q_tokens)

    # ----- normalize TF by length of query -----
    tf_q = Counter(q_tokens)
    #print(tf_q)
    q_len = float(sum(tf_q.values()))
    #print(q_len)

    # weight = (tf/|q|) * idf   (no log TF here; match your doc-length normalization style)
    q_weights = [(w, (tf_q[w] / q_len) * idf_map[w]) for w in tf_q]
    #print(q_weights)
    # tiny query DF to join on "word"
    q_df = spark.createDataFrame(q_weights, ["word", "q_w"])

    # numerator = sum_w (q_w * d_w) per doc
    numer_df = (tfidf_df.join(q_df, on="word", how="inner")
                        .withColumn("prod", F.col("tfidf") * F.col("q_w"))
                        .groupBy("movie_id")
                        .agg(F.sum("prod").alias("numerator")))
    #print(numer_df.show(5))
    # ranking is identical to true cosine since q_norm is a query-constant factor.
    scored = (numer_df.join(doc_norm_df, on="movie_id", how="inner")
                      .filter(F.col("d_norm") > 0)
                      .withColumn("score", F.col("numerator") / F.col("d_norm"))  # no sqrt for q_norm
                      .join(movie_names_df, on="movie_id", how="left")
                      .select("movie_name", "score")
                      .orderBy(F.col("score").desc())
                      .limit(top_k))
    return scored

In [32]:
with open("/content/MovieSummaries/queries.text", "r", encoding="utf-8") as f:
    lines = [ln.strip() for ln in f if ln.strip()]

for line in lines:
    if len(tokenize(line)) == 1:
        print(f"Best match for query word {line}")
        top_docs_for_word_with_name(line.lower(), 10).show(truncate=False)
    else:
        print(f"Best match for query sentence '{line}'")
        run_multi_df(line.lower(), top_k=10).show(truncate=False)


Best match for query word Drama
+------------------------------------+
|movie_name                          |
+------------------------------------+
|Bhakta Prahlada                     |
|Intellectual Property               |
|Johnny Tiger                        |
|Sameera                             |
|Guests Who Arrived on the Last Train|
|Mango Soufflé                       |
|Prince Daewon                       |
|Kismet                              |
|The Eternal Empire                  |
|Nanga Parbat                        |
+------------------------------------+

Best match for query word Romance
+--------------------+
|movie_name          |
+--------------------+
|That Dangerous Age  |
|L'Amour             |
|Ranmuthu Duwa       |
|Lovers and Liars    |
|Moment by Moment    |
|Hollywood Dreams    |
|The Butler's in Love|
|Yeh Dil Aap Ka Huwa |
|Raakh               |
|Coming Back         |
+--------------------+

Best match for query sentence 'Give me the best thriller'
+-----